In [1]:
# 1️⃣  Start in a fresh VM
# (If you've already cloned, delete it first: rm -rf beyond-scale-language-data-diversity)

!git clone https://github.com/alycialee/beyond-scale-language-data-diversity.git
%cd beyond-scale-language-data-diversity

# 2️⃣  Make sure the build tools are modern enough
# %pip install --quiet --upgrade pip setuptools wheel

%pip install pip==24.0

# 3️⃣  Editable-install *into the live kernel*  ← note the %pip magic
%pip install -e .

!pip install -U datasets

fatal: destination path 'beyond-scale-language-data-diversity' already exists and is not an empty directory.
/content/beyond-scale-language-data-diversity
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
Obtaining file:///content/beyond-scale-language-data-diversity
  Preparing metadata (setup.py) ... done
DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063
  Att

In [2]:
%cd beyond-scale-language-data-diversity

[Errno 2] No such file or directory: 'beyond-scale-language-data-diversity'
/content/beyond-scale-language-data-diversity


In [3]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd() / "src"))

In [4]:
sys.path.insert(0, str(pathlib.Path.cwd() / "src" / "diversity"))

See https://www.notion.so/Build-MiniF2F-Proof-Dataset-and-Run-Task2Vec-Embedding-2186c3ca563f80f99ed9d7910950315b

In [5]:
# from torch.utils.data import DataLoader
# from task2vec import Task2Vec
# from datasets import load_dataset
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# import torch


# # ── 1.  Load MiniF2F (488 rows) ────────────────────────────────────────────────
# raw_ds = load_dataset("Tonic/MiniF2F", split="train")        # whole table
# # If you only want the 50 proofs you mentioned, slice or filter here:
# # raw_ds = raw_ds.filter(lambda ex,idx: idx < 50, with_indices=True)

# # ── 2.  Build a single "text" field per row ────────────────────────────────────
# def concat_fields(example):
#     """Merge the human statement + Lean theorem + goal into one string."""
#     pieces = [
#         example["informal_prefix"],        # LaTeX / English statement
#         example["formal_statement"],       # 'theorem …' line
#         example["goal"] # Lean goal state
#         #,example["header"]       # Not including header bc it's mostly the same for all rows
#     ]
#     example["text"] = "\n".join(p for p in pieces if p)
#     return example

# raw_ds = raw_ds.map(concat_fields)

# # ── 3.  Tokenise ───────────────────────────────────────────────────────────────
# tokenizer = GPT2Tokenizer.from_pretrained("distilgpt2")
# tokenizer.pad_token = tokenizer.eos_token        # GPT-2 has no pad token

# def tok_fn(batch):
#     return tokenizer(batch["text"],
#                      padding="longest",
#                      truncation=False)            # truncate long proofs

# tok_ds = raw_ds.map(tok_fn,
#                     batched=True,
#                     remove_columns=raw_ds.column_names)  # keep only encodings
# tok_ds.set_format(type="torch",
#                   columns=["input_ids", "attention_mask"])

# # ── 4.  Embed each solver + probe head ─────────────────────────────────────────
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# probe_A = GPT2LMHeadModel.from_pretrained("distilgpt2").to(device)


# # Pass the dataset directly to embed
# emb_A, _ = Task2Vec(probe_A).embed(tok_ds, epochs=1)


See https://www.notion.so/Implement-Task2Vec-Embedding-Comparison-for-Proof-Solver-Models-2186c3ca563f80a8a35cd6cc370343b4?source=copy_link

In [6]:
%pip install -U bitsandbytes

DEPRECATION: omegaconf 2.0.6 has a non-standard dependency specifier PyYAML>=5.1.*. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of omegaconf or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [7]:
from task2vec import Task2Vec


def embed_no_ft(self, dataset, epochs: int = 0):
    """
    Replacement for Task2Vec.embed that never fine-tunes.
    It only computes the Fisher information and returns (embedding, None).
    """
    self.compute_fisher(dataset)          # needs the dataset!
    embedding = self.extract_embedding(self.model)
    return embedding, None

# ← the important line: no MethodType, just assign the function
Task2Vec.embed = embed_no_ft

In [8]:
import os, gc, torch, warnings, torch.nn.functional as F
from datasets import load_dataset
from transformers import (AutoTokenizer, AutoModelForCausalLM,
                          AutoConfig, BitsAndBytesConfig)
from task2vec import Task2Vec

warnings.filterwarnings("ignore", module="torch")

# ── 1. MiniF2F ────────────────────────────────────────────────────────────
raw_ds = load_dataset("Tonic/MiniF2F", split="train")
raw_ds = raw_ds.map(
    lambda ex: {"text": "\n".join(
        p for p in (ex["informal_prefix"], ex["formal_statement"], ex["goal"]) if p
    )},
    num_proc=4,
)

# ── 2. Models & 8-bit config ──────────────────────────────────────────────
model_ids = [
    "deepseek-ai/DeepSeek-Prover-V2-7B",
    "AI-MO/Kimina-Prover-Preview-Distill-7B",
]

bnb_cfg = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",        # ← add this line
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.float16,   # or torch.bfloat16
)



def make_loss_fn(ignore_id):
    # accepts any extra positional / keyword args Task2Vec passes
    def _loss_fn(logits, tgt, *_, **__):
        # shift-left like Task2Vec expects
        logits = logits[:, :-1, :].contiguous()
        tgt    = tgt[:, 1:].contiguous()
        return F.cross_entropy(
            logits.view(-1, logits.size(-1)),
            tgt.view(-1),
            ignore_index=ignore_id,      # pad/EOS token id – never out of range
        )
    return _loss_fn
embeddings, losses = {}, {}

for mid in model_ids:
    print(f"\n>>> embedding {mid}")

    # ── tokenizer ─────────────────────────────────────────────────────────
    tok = AutoTokenizer.from_pretrained(mid, trust_remote_code=True)
    tok.pad_token = tok.eos_token if tok.pad_token_id is None else tok.pad_token

    tok_ds = raw_ds.map(
        lambda b: tok(b["text"], padding="longest", truncation=True, max_length=2048),
        batched=True,
        remove_columns=raw_ds.column_names,
    )
    tok_ds.set_format(type="torch", columns=["input_ids", "attention_mask"])

    # ── config tweaks ─────────────────────────────────────────────────────
    cfg = AutoConfig.from_pretrained(mid, trust_remote_code=True)
    if cfg.rope_scaling:
        cfg.rope_scaling["factor"]    = float(max(1.0, cfg.rope_scaling["factor"]))
        cfg.rope_scaling["beta_fast"] = float(cfg.rope_scaling["beta_fast"])
        cfg.rope_scaling["beta_slow"] = float(cfg.rope_scaling["beta_slow"])
    cfg.attn_implementation = "sdpa"

    # ── model (8-bit + CPU spill, explicit map) ───────────────────────────
    probe = AutoModelForCausalLM.from_pretrained(
        mid,
        config=cfg,
        quantization_config=bnb_cfg,
        device_map="auto",        # ← keeps as much as fits on GPU-0
        offload_buffers=True,
        low_cpu_mem_usage=True,
        trust_remote_code=True,
    )
    probe.gradient_checkpointing_enable()

    # ── Task2Vec embed ────────────────────────────────────────────────────
    task2vec = Task2Vec(
        probe,
        max_samples=1024,
        loader_opts={"batch_size": 1, "shuffle": True, "num_workers": 0},
    )
    task2vec.loss_fn = make_loss_fn(tok.pad_token_id)

    emb, lm_loss = task2vec.embed(tok_ds, epochs=0)
    embeddings[mid] = torch.tensor(emb.hessian, dtype=torch.float32)
    losses[mid]     = lm_loss

    # ── clean up ──────────────────────────────────────────────────────────
    del probe, tok_ds
    torch.cuda.empty_cache()
    gc.collect()

print("\nDone ✓  (see `embeddings`, `losses`)")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Map (num_proc=4):   0%|          | 0/488 [00:00<?, ? examples/s]


>>> embedding deepseek-ai/DeepSeek-Prover-V2-7B


Map:   0%|          | 0/488 [00:00<?, ? examples/s]

`rope_scaling`'s factor field must be a float >= 1, got 16
`rope_scaling`'s beta_fast field must be a float, got 32
`rope_scaling`'s beta_slow field must be a float, got 1


...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.09G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/139 [00:00<?, ?B/s]

Iteration:   0%|          | 0/488 [00:00<?, ?it/s]


Done ✓  (see `embeddings`, `losses`)


In [18]:
# print results from
embedding_values = list(embeddings.values())

In [19]:
print(embedding_values)

[tensor([5.9605e-08, 0.0000e+00, 0.0000e+00,  ..., 8.1658e-06, 1.9133e-05,
        1.5914e-05]), tensor([2.9445e-05, 0.0000e+00, 0.0000e+00,  ..., 5.9605e-07, 1.2517e-06,
        8.3447e-07])]


In [25]:
cosi = torch.nn.CosineSimilarity(dim=0)

# Get the minimum length
min_len = min(embedding_values[0].size(0), embedding_values[1].size(0))

# Truncate both tensors
a = embedding_values[0][:min_len]
a = torch.clamp(a, min=-1e6, max=1e6)
b = embedding_values[1][:min_len]

# Compute cosine similarity
output = cosi(a, b)
print("\nComputed Cosine Similarity: ", output.item())


Computed Cosine Similarity:  4.5548335947387386e-07


In [26]:
print("Norm of a:", torch.norm(a).item())
print("Norm of b:", torch.norm(b).item())
print("a[:5]:", a[:5])
print("b[:5]:", b[:5])


Norm of a: 1414213.5
Norm of b: 4.9152512550354
a[:5]: tensor([5.9605e-08, 0.0000e+00, 0.0000e+00, 7.1526e-07, 0.0000e+00])
b[:5]: tensor([2.9445e-05, 0.0000e+00, 0.0000e+00, 1.7142e-04, 1.0729e-06])


In [10]:
# # Testing code

# tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
# tokenizer.pad_token = tokenizer.eos_token
# probe_network = GPT2LMHeadModel.from_pretrained("gpt2").to(device)

# dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train", streaming=True).with_format("torch")
# remove_columns = ["text", "timestamp", "url"]

# def preprocess(examples):
#     return tokenizer(examples["text"], return_tensors="pt", padding=True, truncation=True)

# batch_size = 512
# batch = dataset.take(batch_size)
# tokenized_batch = batch.map(preprocess, batched=True, remove_columns=remove_columns)

# embedding, loss = Task2Vec(probe_network).embed(tokenized_batch, epochs=1)

# type(embedding)

# embedding.hessian